### 1. Random Environment with OpenAI Gym

In [32]:
import gym
import random

In [33]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [34]:
actions

2

In [35]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))

Episode:1 Score:17.0 State:[-0.09760849 -0.9583147   0.22045553  1.7246798 ]
Episode:2 Score:29.0 State:[-0.05717931  0.14423113  0.21960442  0.43084922]
Episode:3 Score:52.0 State:[-0.18056385 -0.0539616   0.22452265  0.60949063]
Episode:4 Score:12.0 State:[ 0.17194591  1.2126182  -0.24844031 -2.0318997 ]
Episode:5 Score:32.0 State:[-0.07680852 -0.79925764  0.24224593  1.689557  ]
Episode:6 Score:26.0 State:[-0.02093328  0.7565244  -0.23619372 -1.7513438 ]
Episode:7 Score:33.0 State:[-0.22483066 -0.979734    0.2260173   1.5884702 ]
Episode:8 Score:15.0 State:[ 0.13076587  0.9524917  -0.21051277 -1.6487806 ]
Episode:9 Score:21.0 State:[-0.02257521  0.54484785  0.2116889   0.02104736]
Episode:10 Score:51.0 State:[ 0.10409568  0.2163485  -0.21744558 -0.50758266]


### 2. Create a Deep Learning Model with Keras

In [36]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [37]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states))) # flat node
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [38]:
model = build_model(states, actions)

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### 3. Build Agent with Keras-RL

In [40]:
from rl.agents import DQNAgent # one of the available agents: https://keras-rl.readthedocs.io/en/latest/agents/overview/
from rl.policy import BoltzmannQPolicy # vs. value-based RL
from rl.memory import SequentialMemory

In [41]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy() # error function?
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [42]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae']) # mean average error
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 10:25 - reward: 1.0000

D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 4482/10000 [============>.................] - ETA: 24s - reward: 1.0000

Exception ignored in: <function Viewer.__del__ at 0x00000287FBFFA940>
Traceback (most recent call last):
  File "D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "D:\Python\Projects\AI_Bootcamp\TensorFlow\tf\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x00000287FBFAAD60; to 'Win32Window' at 0x00000287FBA64190>


10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000: 0s - r
116 episodes - episode_reward: 85.121 [10.000, 200.000] - loss: 2.782 - mae: 18.078 - mean_q: 36.616

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: 1.0000
51 episodes - episode_reward: 195.706 [161.000, 200.000] - loss: 7.196 - mae: 39.610 - mean_q: 80.106

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 46s 5ms/step - reward: 1.0000
51 episodes - episode_reward: 197.157 [161.000, 200.000] - loss: 7.944 - mae: 44.441 - mean_q: 89.585

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 198.980 [182.000, 200.000] - loss: 6.575 - mae: 43.551 - mean_q: 87.623

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 45s 4ms/step - reward: 1.0000
done, took 223.724 seconds


In [45]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 175.000, steps: 175
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
197.5


In [47]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 185.000, steps: 185
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 198.000, steps: 198
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


### 4. Reload Agent from Memory

In [48]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [49]:
del model
del dqn
del env

In [51]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [52]:
dqn.load_weights('dqn_weights.h5f')

In [54]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
